# HW4: QA Agent

## Dependencies and LLM Backbone

In [ ]:
!pip install langchain==1.0.5
!pip install langchain-core
!pip install langchain-community
!pip install faiss-cpu
!pip install kagglehub

In [ ]:
# This is the list of countries we are using (with their official languages)
# Feel free to use it in your code
list_of_countries = {}
with open("countries_with_languages.tsv", "r"  ) as f:
    for line in f.readlines():
        country, langs = line.strip().split("\t")
        list_of_countries[country] = langs.split(",")

### Choice 1: OpenAI API

The notebook's implementation is based on this.
Feel free to change the model, and please keep track of your usage on the "Usage" page on [LiteLLM API webpage](https://ai-gateway.andrew.cmu.edu/ui/).

In [ ]:
!pip install langchain-openai

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import getpass, os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")
openai_model_id = "gpt-4o-mini-2024-07-18"
openai_embmodel_id = "azure/text-embedding-3-small"

llm = ChatOpenAI(
    model=openai_model_id,
    api_key=os.environ["OPENAI_API_KEY"],
    base_url="https://ai-gateway.andrew.cmu.edu/"
)
embeddings = OpenAIEmbeddings(
    model=openai_embmodel_id,
    api_key=os.environ['OPENAI_API_KEY'],
    base_url='https://ai-gateway.andrew.cmu.edu/'
)

### Choice 2: Hugging Face Models

You may also use Hugging Face models without API credits if you have available GPU resource. You might have to the change prompt templates according to your model choice.

In [ ]:
!pip install langchain-huggingface text-generation transformers google-search-results 
!pip install numexpr langchainhub sentencepiece jinja2 bitsandbytes accelerate

In [ ]:
import getpass, os
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline, HuggingFaceEmbeddings

os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your Hugging Face API key: ")
hgf_model_id = "Qwen/Qwen3-0.6B"
hgf_embmodel_id = "sentence-transformers/all-mpnet-base-v2"

hgf_model = HuggingFacePipeline.from_model_id(
    model_id=hgf_model_id,
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=128,
        do_sample=False,
    ),
)
llm = ChatHuggingFace(hgf_model)
embeddings = HuggingFaceEmbeddings(model_name=hgf_embmodel_id)

## Handling different type of questions

Implement the answer formatting and extraction for each question type. You may change the prompt to fit your processing function.

In [ ]:
from langchain.agents import create_agent

### 🗺️Global Trekker

In [ ]:
global_trekker_messages = [
    {"role": "system", "content": "You are an expert in world knowledge."},
    {"role": "user", "content": "Given the following paragraph, guess the most likely country and city. Answer in the format of [country], [city]."},
]
global_trekker = create_agent(model=llm, tools=[])

In [ ]:
def extract_global_trekker_answer(response: str) -> tuple[str, str]:
    # TODO: Extract the country and city from the response
    # return country, city
    pass

### 🍽️Culinary Detective

In [ ]:
import kagglehub
import pandas as pd

def gather_recipe_data(kaggledataset: str) -> list[str]:
    dataset_path = kagglehub.dataset_download(kaggledataset)
    df = pd.read_csv(f"{dataset_path}/Receipes from around the world.csv", encoding='latin-1')
    # TODO: Process the dataframe to list of text entries for retrieval
    # This is a basic implementation
    recipes = ["; ".join(map(str, row)) for row in df.values.tolist()]
    return recipes

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.tools import tool

recipes = gather_recipe_data("prajwaldongre/collection-of-recipes-around-the-world")
docs = [Document(page_content=recipe) for recipe in recipes]
vector = FAISS.from_documents(docs, embeddings)
retriever = vector.as_retriever(search_kwargs={"k": 2})

In [ ]:
@tool
def retrieve_recipes(query: str):
  """
  Retrieves recipes based on a search query.
  """
  return retriever.invoke(query)

culinary_detective_messages = [
    {"role": "system", "content": "You are an expert in world cuisines and culinary traditions."},
    {"role": "human", "content": "Given the following description, guess the most likely country and region. Answer in the format of [country], [region]."},
]
culinary_detective = create_agent(model=llm, tools=[retrieve_recipes])

In [ ]:
def extract_culinary_detective_answer(response: str) -> tuple[str, str]:
    # TODO: Extract the country and region from the response
    # return country, region
    pass

### 👄Lingua Locale

In [ ]:
lingua_locale_messages = [
    {"role": "system", "content": "You are an expert in languages and cultural nuances."},
    {"role": "user", "content": "Given the following sentence, guess the which country's website it is from. Answer in the format of [country]."},
]
lingua_locale = create_agent(model=llm, tools=[])

In [ ]:
def extract_lingua_locale_answer(response: str) -> tuple[str, str]:
    # TODO: Extract the country and "none" from the response
    # only the first field is used, the second is a dummy field to make the return type consistent
    # return country, "none"
    pass

## Answering questions
This part includes how we load the questions and generate the prediction in desired format. 

In [ ]:
def geoguesser(q: dict, print_raw_response=False) -> tuple[str, str]:
    if q["type"] == "GlobalTrekker":
        query = {"role": "user", "content": f"Paragraph: {q["paragraph"]}"}
        messages, agent, extractor = global_trekker_messages, global_trekker, extract_global_trekker_answer
    elif q["type"] == "CulinaryDetective":
        query = {"role": "user", "content": f"Ingredients: {q["ingredient"]}. Description: {q["description"]}"}
        messages, agent, extractor = culinary_detective_messages, culinary_detective, extract_culinary_detective_answer
    else: #q["type"] == "LinguaLocale":
        query = {"role": "user", "content": f"Sentence: {q["sentence"]}"}
        messages, agent, extractor = lingua_locale_messages, lingua_locale, extract_lingua_locale_answer

    response_all = agent.invoke({"messages": messages + [query]})
    response = response_all["messages"][len(messages)+1].content
    if print_raw_response: print(f"{q["type"]}: {response_all}")
    return extractor(response)

In [ ]:
import json

# Here, we load the examples questions. Public/private set will be in the same format
dataset_name = "example.jsonl"
questions = []
with open(dataset_name, "r") as f:
    for line in f.readlines():
        questions.append(json.loads(line))

In [ ]:
# Test run on one question
# You might want to save the raw response for debugging answer formatting/extraction
# If the extracted answer seems off, check the raw response instead of running inference repeatedly
geoguesser(questions[0], print_raw_response=True)

In [ ]:
# Sample script to generate answers
answers = []
for q in questions:
    country, category = geoguesser(q)
    answers.append(f"{q["type"]}\t{country}\t{category}")

with open("example_output.txt", "w") as f:
    for answer in answers:
        f.write(answer + "\n")

## Evaluation
This is how we calculate the scores on Gradescope (details subject to change, but the general logic will stay the same).

In [ ]:
def soft_match(answer, expectedAnswer):
    score = 0.0
    if expectedAnswer in answer:
        score = len(expectedAnswer) / len(answer)
    return score

def exact_match(answer, expectedAnswer):
    score = 0.0
    if expectedAnswer == answer:
        score = 1.0
    return score

In [ ]:
answers = []
for q in questions:
    answers.append((q["type"], q["country"], q.get("city", q.get("region", ""))))
with open("example_output.txt", "r") as f:
    preds = [line.split("\t") for line in f.readlines()]

scores = {"GlobalTrekker": [], "CulinaryDetective": [], "LinguaLocale": []}
for (q_type, exp_country, exp_place), (p_type, pred_country, pred_place) in zip(answers, preds):
    assert q_type == p_type
    country_score = soft_match(pred_country, exp_country)
    category_score = 0.0
    weights = [0.0, 0.0]
    if q_type == "GlobalTrekker":
        #  correct country -> 80%, correct counrry and city -> +20%
        weights = [0.8, 0.2]
        if exp_place != "" and country_score > 0:
            category_score = soft_match(pred_place, exp_place)
    elif q_type == "CulinaryDetective":
        # correct country -> 60%, correct country and region -> +40%
        weights = [0.6, 0.4]
        if exp_place != "All" and country_score > 0:
            category_score = exact_match(pred_place, exp_place)
    else: # LinguaLocale
        # correct country -> 60%, matched official language -> +40%
        weights = [0.6, 0.4]
        if country_score > 0:
            category_score = 1.0
        else: # incorrect country. language match works only if pred_country is a clean answer
            exp_langs = list_of_countries.get(exp_country, [])
            pred_langs = list_of_countries.get(pred_country, [])
            if any(lang in exp_langs for lang in pred_langs):
                category_score = 1.0

    score = weights[0] * country_score + weights[1] * category_score
    scores[q_type].append(score)

for q_type, score_list in scores.items():
    avg_score = sum(score_list) / len(score_list)
    print(f"{q_type} Average Score: {avg_score:.4f}")